# Detección de crecimiento urbano con imagenes del satélite Sentinel - 1

## 2_ Predicción

Este notebook genera la prediccion del modelo sobre un nuevo dataset. Para ello primero descargamos las imagenes satelitales y luego con satproc generamos el dataset de predicción con el formato necesario para el modelo de ML


**Definicion de los parametros para la predicción**

In [1]:
from unetseg.predict import PredictConfig, predict
from unetseg.evaluate import plot_data_results
import subprocess
import os
import pandas as pd
import glob

2022-06-14 13:06:04.619777: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-06-14 13:06:04.619817: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
CSV_RESULTS_FILE = "buckets_v3_1_resultados.csv"
CSV_IMG_S2_FILE  = "buckets_v3_1_images-s1.csv"
    
BANDS = "VHd_VVd_VHdVVd"
YEAR  = "2021"
BASE_PATH = "../"

#model
SIZE      = 100
STEP_SIZE = 100

MODEL = f'UNet_TEST_160x160_spe100_3N_spe100_colombia_atlantico.h5'


CONDITION = True
ZONA = "pacifico"

### Descarga de las imagenes de predicción

Descarga las imagenes del bucket

### Satproc

Con esta herramienta genereamos un dataset de imágenes de predicción que, a diferencia del de entrenamiento, solo contiene imágenes.

In [3]:
def execute_satproc_function(SIZE,STEP_SIZE,BASE_PATH,img_folder,YEAR,DPTO_COD,COD_MUNICIPIO):
   
   
    path_to_files  = f'{img_folder}/{COD_MUNICIPIO}/*.tif'
    dataset_folder_ = f'{BASE_PATH}/dataset/data_predict/province_{COD_MUNICIPIO}_{DPTO_COD}_year{YEAR}/{str(SIZE)}_{str(STEP_SIZE)}/'
  
    print("path_to_files-->",path_to_files)
    print("dataset_folder_-->",dataset_folder_)
   
    !satproc_extract_chips $path_to_files \
                    -o $dataset_folder_ \
                    --size $SIZE \
                    --step-size $STEP_SIZE \
                    --rescale \
                    --rescale-mode values --min -15 --max 5
    return dataset_folder_

Pasamos la ruta de las imágenes de predicción y la ruta donde queremos que se guarde el resultado.



## Prediccion

Definimos la configuración para la predicción. Debemos pasar la ruta de las imágenes en el dataset de predicción, que es la ruta de salida de la línea anterior, y la correspondiente al modelo que entrenamos.

In [10]:
def excetute_prediction (dataset_folder,BASE_PATH,YEAR,SIZE,STEP_SIZE,MODEL,DPTO_COD,COD_MUNICIPIO):
    predict_config = PredictConfig(
         
            images_path   = dataset_folder, # ruta a las imagenes sobre las cuales queremos predecir
            results_path  = f'{BASE_PATH}/dataset/data_results/province_{COD_MUNICIPIO}_{DPTO_COD}_year{YEAR}/{str(SIZE)}_{str(STEP_SIZE)}/'
, # ruta de destino para nuestra predicción
            batch_size  = 16,
            model_path  = os.path.join(BASE_PATH +'data/weights', MODEL),  #  ruta al modelo (.h5)
            height = 160,
            width  = 160,
            n_channels = 3,
            n_classes  = 1,
            class_weights = [1])

    #A continuación ejecutamos la predicción
    predict(predict_config)    
    #plot_data_results(num_samples=2, fig_size=(2, 2), predict_config=predict_config, img_ch =2, n_bands=3)

    #create VTR
    !rm tmp_list.txt

    vrt_path        = f'{predict_config.results_path}/../{COD_MUNICIPIO}_{YEAR}.vrt' # ruta de destino para nuestra predicción
    tif_path        = f'{predict_config.results_path}/../{COD_MUNICIPIO}_{YEAR}.tif'
    predict_results = f'{predict_config.results_path}/*tif' # ruta de destino para nuestra predicción

    filenames = glob.glob(predict_results)

    with open('tmp_list.txt', 'w') as f:
        for line in filenames:
            f.write(line)
            f.write('\n')
    !gdalbuildvrt -input_file_list tmp_list.txt $vrt_path
    
    !gdal_translate -of GTiff -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=9 $vrt_path $tif_path
    
    return tif_path
    

## RUN

In [9]:
# load the data with pd.read_csv
df = pd.read_csv(CSV_IMG_S2_FILE).reset_index()
df_results = pd.read_csv(CSV_RESULTS_FILE).reset_index()

df.sort_values(by=["PRIORIDAD"], ascending=True)

if CONDITION:
   
    df = df[df["REGION"] == ZONA]


    
for index, row in df.iterrows():
   
    COD_MUNICIPIO = row['COD_MUNICIPIO']
    NOM_MUNICIPIO = row['NOM_MUNICIPIO']
    DPTO_COD      = row['DPTO_COD']
    DPTO_NAME     = row['DPTO_NAME']
    PRIORIDAD     = row['PRIORIDAD']
    REGION        = row['REGION']
    BUCKET_IMG    = row['BUCKET']
    
    print ('=================================================================================================')
    print ( f'Working with {NOM_MUNICIPIO} - {DPTO_NAME}, corresponding with COD_MUNICIPIO {COD_MUNICIPIO} and DPTO_COD {DPTO_COD}' )
    print ('=================================================================================================')

    print ( f'priority number: {PRIORIDAD}')
    print ( f'REGION : {REGION}')
    ############################
    ## Download images
    ###########################
    print ("Download images...")
    img_folder = f'{BASE_PATH}/images_s1_{BANDS}/{YEAR}/'
    !mkdir -p $img_folder
    !gsutil -m cp -r -n $BUCKET_IMG  $img_folder
    
    ############################
    ## Satproc  & predict 
    ###########################
    
    #read bucket
    BUCKET_RESULTS = df_results[ df_results["COD_MUNICIPIO"] == COD_MUNICIPIO].BUCKET
    backet_r = BUCKET_RESULTS.values[0]
    print (backet_r)
    
    #satproc
    dataset_folder = execute_satproc_function(SIZE,STEP_SIZE,BASE_PATH,img_folder,YEAR,DPTO_COD,COD_MUNICIPIO)
    #predict
    TIF_path = excetute_prediction (dataset_folder,BASE_PATH,YEAR,SIZE,STEP_SIZE,MODEL,DPTO_COD,COD_MUNICIPIO)
    
    # upload to a bucket
    !gsutil -m cp -r  $TIF_path $backet_r

----------------- pacifico
    index  COD_MUNICIPIO           NOM_MUNICIPIO  DPTO_COD        DPTO_NAME  \
10     10          19455                 Miranda        19            Cauca   
11     11          19001                 Popayan        19            Cauca   
12     12          19698  Santander De Quilichao        19            Cauca   
16     16          27361                 Istmina        27            Choco   
17     17          27001                  Quibdo        27            Choco   
31     31          52051                Arboleda        52           Nariño   
32     32          52079               Barbacoa         52           Nariño   
33     33          52240               Chachagui        52           Nariño   
34     34          52207                 Consaca        52           Nariño   
35     35          52250              El charco         52           Nariño   
36     36          52356                 Ipiales        52           Nariño   
37     37          52390 